## Task 1: Setup the dataset

- Load the MNIST dataset using the Hugging Face datasets library.
- Convert the image data into Numpy arrays and normalize pixel values to the range [0,1].
- Flatten each image into a vector of 784 features.
- Split the dataset into training and testing sets.
- Randomly select an initially labeled dataset of 200 samples from training samples.
- Generate an "Unlabeled Pool," the Initial Dataset excluding 200 samples.

In [1]:
import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/home/samarth/.local/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = datasets.load_dataset("mnist")

In [3]:
# convert the image data into a numpy array and normalize the values from 0 to 1
X = np.array(data['train']["image"]) / 255
y = np.array(data['train']["label"])


In [4]:
X = X.reshape(X.shape[0], -1)
X.shape

(60000, 784)

In [5]:
X_test = np.array(data['test']["image"]) / 255
y_test = np.array(data['test']["label"])

# flatten the test data
X_test = X_test.reshape(X_test.shape[0], -1)

In [11]:
# randomly select 200 samples from training dataset and create a labelled dataset
np.random.seed(45)   
idx = np.random.choice(X.shape[0], 200, replace=False)
X_train_labelled = X[idx]
y_train_labelled = y[idx]

In [12]:
# create a pool of unlabelled data
X_train_unlabelled = np.delete(X, idx, axis=0)
y_train_unlabelled = np.delete(y, idx, axis=0)

X_train_unlabelled.shape


(59800, 784)

## Task 2: Implement Random Sampling for Active Learning


- Train a Random Forest Classifier (you can use “from sklearn.ensemble import RandomForestClassifier”)  on the initial dataset of 200 samples.
- Implement an active learning loop for 20 iterations:
    - Randomly select a sample from the unlabeled pool.
    - Get the selected sample and its true label.
    - Add the sample and label to the labeled dataset.
    - Remove the selected sample and label from the pool.
    - Retrain the model on the updated dataset.
    - Check the model's accuracy on the test set.
    - Print accuracy after every iteration.


In [13]:
from sklearn.ensemble import RandomForestClassifier

# train a random forest classifier on the labelled data
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_labelled, y_train_labelled)

# predict the unlabelled data
y_prediction = clf.predict(X_test)

# calculate the accuracy of the classifier
accuracy = np.mean(y_test == y_prediction)
accuracy

0.7777

In [ ]:
# implementing active learning by for 20 iterations by randomly choosing the samples from the unlabelled pool.
task1_results = []
for i in range(20):
    idx_curr = np.random.choice(X_train_unlabelled.shape[0], 1, replace=False) # randomly select 1 sample

    # add the sample to the labelled dataset
    X_train_labelled = np.concatenate([X_train_labelled, X_train_unlabelled[idx_curr]])
    y_train_labelled = np.concatenate([y_train_labelled, y_train_unlabelled[idx_curr]])

    # remove the sample from the unlabelled dataset
    X_train_unlabelled = np.delete(X_train_unlabelled, idx_curr, axis=0)
    y_train_unlabelled = np.delete(y_train_unlabelled, idx_curr, axis=0)

    # retrain the classifier
    clf.fit(X_train_labelled, y_train_labelled)

    # predict the unlabelled data
    y_prediction= clf.predict(X_test)

    # calculate the accuracy of the classifier
    accuracy = np.mean(y_test == y_prediction)
    task1_results.append(accuracy)
    print(f"Iteration: {i} Accuracy: {accuracy}")


Iteration: 0 Accuracy: 0.7705
Iteration: 1 Accuracy: 0.7791
Iteration: 2 Accuracy: 0.7682
Iteration: 3 Accuracy: 0.7717
Iteration: 4 Accuracy: 0.776
Iteration: 5 Accuracy: 0.7863
Iteration: 6 Accuracy: 0.7662
Iteration: 7 Accuracy: 0.7782
Iteration: 8 Accuracy: 0.7798
Iteration: 9 Accuracy: 0.7889
Iteration: 10 Accuracy: 0.7819
Iteration: 11 Accuracy: 0.7795
Iteration: 12 Accuracy: 0.7835
Iteration: 13 Accuracy: 0.7845
Iteration: 14 Accuracy: 0.7868
Iteration: 15 Accuracy: 0.7984
Iteration: 16 Accuracy: 0.8022
Iteration: 17 Accuracy: 0.7971
Iteration: 18 Accuracy: 0.7862
Iteration: 19 Accuracy: 0.7971


## Task 3: Implement Uncertainty Sampling for Active Learning.

- Train a Random Forest Classifier (you can use “from sklearn.ensemble import RandomForestClassifier”)  on the initial dataset of 200 samples.
- Implement an active learning loop for 20 iterations:
    - Compute uncertainty (Label Entropy) for each sample in the unlabeled pool using entropy.
    - Select the sample with the highest uncertainty and query its true label.
    - Add the queried sample to the labelled dataset and remove it from the unlabelled pool.
    - Retrain the model and check the model's accuracy on the test set.
    - Print accuracy after every iteration


In [ ]:
np.random.seed(45)   
idx = np.random.choice(X.shape[0], 200, replace=False)
X_train_labelled = X[idx]
y_train_labelled = y[idx]